### Question 1. dlt Version

In [ ]:
# !pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"

In [1]:
# Q: What's the version of dlt that you installed?

import dlt
print(dlt.__version__)

1.13.0


In [ ]:
import requests
import pandas as pd

In [3]:
# Step 1: Create DLT resource
@dlt.resource(write_disposition="replace", name="zoomcamp_data")
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

### Question 2. dlt pipeline

In [4]:
from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)

In [5]:
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Run started at 2025-07-12 09:47:48.176235+00:00 and COMPLETED in 9.93 seconds with 4 steps.
Step extract COMPLETED in 0.63 seconds.

Load package 1752313671.0163646 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.08 seconds.
Normalized data for the following tables:
- zoomcamp_data: 948 row(s)
- _dlt_pipeline_state: 1 row(s)

Load package 1752313671.0163646 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 6.39 seconds.
Pipeline zoomcamp_pipeline load step completed in 6.37 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /workspaces/llm-zoomcamp-2025-homework/Workshop/db.qdrant location to store data
Load package 1752313671.0163646 is LOADED and contains no failed jobs

Step run COMPLETED in 9.93 seconds.
Pipeline zoomcamp_pipeline load step completed in 6.37 seconds
1 load package(s) were loade

In [ ]:
# Q: How many rows were inserted into the zoomcamp_data collection?

# Hint: Look for "Normalized data for the following tables:" in the trace output.

# A: Normalized data for the following tables: - zoomcamp_data: 948 row(s)

### Question 3. Embeddings

In [ ]:
# Q: When inserting the data, an embedding model was used. Which one?

# A: fast-bge-small-en

In [ ]:
# Hint:
# You can find this out by inspecting the meta.json file created in the target folder.
# During the data insertion process, a folder named db.qdrant will be created,
# and the meta.json file will be located inside this folder.